In [1]:
import cv2
import mediapipe as mp
import os

# Function to overlay a skeleton on a person detected in a video.
def process_video(video_path, file_name, save_path):
    # Initialize MediaPipe drawing utilities and holistic model for pose detection.
    mp_drawing = mp.solutions.drawing_utils
    mp_holistic = mp.solutions.holistic

    # Specify the thickness and color of the skeleton here
    skeleton_thickness = 10  # Increase this value to make the skeleton lines thicker
    circle_diameter = 6  # Increase this value to make the landmark circles larger
    color = (0, 255, 0)  # Skeleton color (RGB)

    # Create DrawingSpec for landmarks and connections
    landmark_spec = mp_drawing.DrawingSpec(color=color, thickness=circle_diameter, circle_radius=circle_diameter)
    connection_spec = mp_drawing.DrawingSpec(color=color, thickness=skeleton_thickness)

    # List of body parts that MediaPipe Holistic can detect and will be extracted.
    # Detailed list of body parts for reference, not directly used in the drawing.
    body_parts = [
        'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
        'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
        'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
        'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW',
        'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY',
        'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB',
        'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE',
        'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL',
        'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
    ] 

    # Combine the video path and file name to form the full path and attempt to open the video.
    video_full_path = os.path.join(video_path, file_name)
    cap = cv2.VideoCapture(video_full_path)
    if not cap.isOpened():
        print(f"Failed to open video file {video_full_path}.")
        return

    # Retrieve video dimensions and prepare for output video creation.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    # Generate a new file name for the output to preserve the original video.
    base_file_name, file_extension = os.path.splitext(file_name)
    output_file_name = f"{base_file_name}_processed{file_extension}"
    output_file = os.path.join(save_path, output_file_name)
    
    # Define video codec and create a VideoWriter object to save the processed video.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, 20.0, (frame_width, frame_height))

    # Process each frame to detect and overlay a skeleton.
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break  # End loop if there are no more frames to process.

            # Convert frame to RGB for MediaPipe processing and disable writing to improve performance.
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False    
            results = holistic.process(image)

            # Re-enable writing and convert back to BGR for OpenCV processing.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Draw the detected pose landmarks on the frame.
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=landmark_spec, 
                connection_drawing_spec=connection_spec
            )
            out.write(image)  # Save the processed frame to the output video file.

            # Allow exiting the loop with the ESC key.
            if cv2.waitKey(5) & 0xFF == 27:
                break

    # Release the video capture and writer objects and close all OpenCV windows.
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Function to process all videos in a specified root folder and its subdirectories.
def process_all_videos_in_folder(root_folder, save_path):
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            # Filter to process only MP4 and AVI video files.
            if file.endswith(('.mp4', '.avi')):
                video_full_path = os.path.join(subdir, file)
                print(f"Processing {video_full_path}")
                # Process each video, saving processed versions in the same directory as the original.
                process_video(subdir, file, subdir)

if __name__ == "__main__":
    # Specify the directory of videos to process and where to save the processed versions.
    root_folder = 'sample videos'  # Placeholder for the directory containing videos to be processed.
    save_path = root_folder  # Processed videos will be saved in the same directory as their originals.
    process_all_videos_in_folder(root_folder, save_path)